In [ ]:
# Cell 1: Install libraries
!pip install torch_sparse
!pip install torch_scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_sparse: filename=torch_sparse-0.6.18-cp311-cp311-linux_x86_64.whl size=2847711 sha256=da04e0b8b84680834c3405df36f4eb2052aadb6f84db86939977d13ef9d75528
  Stored in directory: /root/.cache/pip/wheels/75/e2/1e/299c596063839303657c211f587f05591891cc6cf126d94d21
Successfully built torch_sparse
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=3622745 sha256=5d2bcb00030e27c328231da1c4d0ef15abd7722160f65a6857d16d892eb033cd
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch_scatter


In [ ]:
# Cell 2: Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch_sparse import SparseTensor, matmul
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import scipy.sparse as sp
import math
from sklearn.metrics import roc_auc_score
import time
import os

# Enable CUDA optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [ ]:
# Cell 3: Data Loader
class PreprocessedDataLoader:
    def __init__(self, data_dir='/kaggle/input/dkr2025/'):
        self.data_dir = data_dir

    def load_preprocessed_data(self):
        """Load preprocessed data"""
        print("Loading preprocessed data...")

        # Load sparse matrices
        train_matrix = sp.load_npz(self.data_dir + 'train_matrix.npz')
        test_matrix = sp.load_npz(self.data_dir + 'test_matrix.npz')
        adj_matrix = sp.load_npz(self.data_dir + 'adj_matrix.npz')

        # Load mappings
        user_to_idx = np.load(self.data_dir + 'user_to_idx.npy', allow_pickle=True).item()
        movie_to_idx = np.load(self.data_dir + 'movie_to_idx.npy', allow_pickle=True).item()

        # Load DataFrame with ratings
        train_ratings = pd.read_csv(self.data_dir + 'train_ratings.csv')
        test_ratings = pd.read_csv(self.data_dir + 'test_ratings.csv')

        print("Data successfully loaded!")
        print(f"Users: {len(user_to_idx):,}")
        print(f"Movies: {len(movie_to_idx):,}")
        print(f"Training interactions: {len(train_ratings):,}")
        print(f"Test interactions: {len(test_ratings):,}")

        return {
            'train_matrix': train_matrix,
            'test_matrix': test_matrix,
            'adj_matrix': adj_matrix,
            'user_to_idx': user_to_idx,
            'movie_to_idx': movie_to_idx,
            'train_ratings': train_ratings,
            'test_ratings': test_ratings
        }

In [ ]:
# Cell 4: Dataset with 1 negative example
class MovieLensDataset(Dataset):
    def __init__(self, ratings_df, num_users, num_movies):
        """Dataset for LightGCN training with 1 negative example"""
        # Take only positive interactions (label=1)
        positive_ratings = ratings_df[ratings_df['label'] == 1].copy()

        # Create interaction set for quick checking
        interactions_set = set(zip(positive_ratings['user_idx'], positive_ratings['movie_idx']))

        self.users = positive_ratings['user_idx'].values
        self.items = positive_ratings['movie_idx'].values
        self.num_users = num_users
        self.num_movies = num_movies
        self.interactions_set = interactions_set

        # Prepare popular movies for better negative sampling
        item_counts = positive_ratings['movie_idx'].value_counts()
        self.popular_items = item_counts.index.values[:10000]  # 10k popular movies

        print(f"Positive interactions: {len(self.users):,}")
        print(f"Using 1 negative example per positive")

    def __len__(self):
        return len(self.users)

    def sample_negative_item(self, user, pos_item):
        """Sample one negative item"""
        attempts = 0
        while attempts < 100:
            # Sample from popular movies with 70% probability
            if np.random.random() < 0.7 and len(self.popular_items) > 0:
                neg_item = np.random.choice(self.popular_items)
            else:
                neg_item = np.random.randint(0, self.num_movies)

            # Verify this is truly a negative interaction
            if neg_item != pos_item and (user, neg_item) not in self.interactions_set:
                return neg_item
            attempts += 1

        # If not found, return random (not equal to positive)
        neg_item = np.random.randint(0, self.num_movies)
        while neg_item == pos_item:
            neg_item = np.random.randint(0, self.num_movies)
        return neg_item

    def __getitem__(self, idx):
        user = self.users[idx]
        pos_item = self.items[idx]

        # Sample ONE negative item
        neg_item = self.sample_negative_item(user, pos_item)

        return {
            'user': torch.tensor(user, dtype=torch.long),
            'pos_item': torch.tensor(pos_item, dtype=torch.long),
            'neg_item': torch.tensor(neg_item, dtype=torch.long)  # Only 1!
        }

In [ ]:
# Cell 5: LightGCN model (fixed according to recommendations)
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64, num_layers=3):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        # User and item embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Initialization as in original LightGCN
        nn.init.normal_(self.user_embedding.weight, std=0.1)
        nn.init.normal_(self.item_embedding.weight, std=0.1)

        # NOT using layer weights (simple average in original)

    def forward(self, adjacency_matrix):
        # Initial embeddings
        user_emb = self.user_embedding.weight
        item_emb = self.item_embedding.weight

        # Concatenate for graph propagation
        all_embeddings = torch.cat([user_emb, item_emb], dim=0)
        embeddings_per_layer = [all_embeddings]

        # Propagation through layers (LightGCN propagation)
        for _ in range(self.num_layers):
            all_embeddings = matmul(adjacency_matrix, all_embeddings)
            embeddings_per_layer.append(all_embeddings)

        # Weighted sum of all layer embeddings (simple average)
        final_embeddings = torch.stack(embeddings_per_layer, dim=0).mean(dim=0)

        # Split back into users and items
        final_user_emb, final_item_emb = torch.split(
            final_embeddings, [self.num_users, self.num_items]
        )

        return final_user_emb, final_item_emb

In [ ]:
# Cell 6: BPRLoss WITHOUT regularization
class BPRLoss(nn.Module):
    def __init__(self):
        super().__init__()
        # NO lambda_reg! LightGCN doesn't use embedding regularization

    def forward(self, user_emb, pos_item_emb, neg_item_emb):
        """Pure BPR Loss as in original LightGCN"""
        # Calculate scores
        pos_scores = (user_emb * pos_item_emb).sum(dim=1)
        neg_scores = (user_emb * neg_item_emb).sum(dim=1)

        # BPR loss without regularization
        loss = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8).mean()

        return loss

In [ ]:
# Cell 7: Metrics Calculator with Precision@K and Recall@K
class MetricsCalculator:
    def __init__(self, train_matrix, test_matrix, num_users, num_items, device, k=10):
        self.train_matrix = train_matrix.tolil()
        self.test_matrix = test_matrix.tolil()
        self.num_users = num_users
        self.num_items = num_items
        self.device = device
        self.k = k

        # Cache training items for quick access
        self.train_items = {}
        for user in range(num_users):
            self.train_items[user] = set(self.train_matrix.rows[user])

    def calculate_all_metrics(self, model, adjacency_matrix, batch_size=4096, roc_samples=10000):
        """Calculate all metrics: HR@K, NDCG@K, Precision@K, Recall@K, ROC-AUC"""
        model.eval()

        with torch.no_grad():
            user_emb, item_emb = model(adjacency_matrix)

        # Get users with test interactions
        users_with_test = [
            u for u in range(self.num_users)
            if len(self.test_matrix.rows[u]) > 0
        ]

        print(f"Calculating metrics for {len(users_with_test)} users...")

        hr_list, ndcg_list, precision_list, recall_list = [], [], [], []

        # Batch processing for HR, NDCG, Precision, Recall
        for start_idx in tqdm(range(0, len(users_with_test), batch_size),
                             desc="Calculating ranking metrics"):
            end_idx = min(start_idx + batch_size, len(users_with_test))
            batch_users = users_with_test[start_idx:end_idx]

            if not batch_users:
                continue

            # User embeddings for batch
            batch_user_emb = user_emb[batch_users]

            # Scores for all items
            scores = torch.matmul(batch_user_emb, item_emb.T)

            # Mask training items
            for i, user in enumerate(batch_users):
                train_indices = list(self.train_items[user])
                if train_indices:
                    scores[i, train_indices] = -float('inf')

            # Get top-K predictions
            _, topk_indices = torch.topk(scores, k=self.k, dim=1)

            # Calculate metrics for each user in batch
            for i, user in enumerate(batch_users):
                test_items = self.test_matrix.rows[user]
                if not test_items:
                    continue

                # Take all test items as relevant
                relevant_items = set(test_items)
                predicted_items = set(topk_indices[i].cpu().numpy())

                # Hit Rate
                hr = 1 if predicted_items.intersection(relevant_items) else 0
                hr_list.append(hr)

                # NDCG
                dcg = 0.0
                for rank, item in enumerate(topk_indices[i].cpu().numpy(), 1):
                    if item in relevant_items:
                        dcg += 1 / math.log2(rank + 1)

                # Ideal DCG (IDCG)
                idcg = sum(1 / math.log2(r + 1) for r in range(1, min(len(relevant_items), self.k) + 1))
                ndcg = dcg / idcg if idcg > 0 else 0
                ndcg_list.append(ndcg)

                # Precision@K
                hits = len(predicted_items.intersection(relevant_items))
                precision = hits / self.k
                precision_list.append(precision)

                # Recall@K
                recall = hits / len(relevant_items) if len(relevant_items) > 0 else 0
                recall_list.append(recall)

        # Calculate ROC-AUC
        roc_auc = self._calculate_roc_auc(user_emb, item_emb, num_samples=roc_samples)

        # Average metrics
        metrics = {
            f'hr@{self.k}': np.mean(hr_list) if hr_list else 0.0,
            f'ndcg@{self.k}': np.mean(ndcg_list) if ndcg_list else 0.0,
            f'precision@{self.k}': np.mean(precision_list) if precision_list else 0.0,
            f'recall@{self.k}': np.mean(recall_list) if recall_list else 0.0,
            'roc_auc': roc_auc
        }

        return metrics

    def _calculate_roc_auc(self, user_emb, item_emb, num_samples=10000):
        """Calculate ROC-AUC"""
        scores = []
        labels = []

        # Sample users
        users_with_test = [
            u for u in range(self.num_users)
            if len(self.test_matrix.rows[u]) > 0
        ]

        num_users_to_sample = min(num_samples // 2, len(users_with_test))
        sampled_users = np.random.choice(users_with_test, size=num_users_to_sample, replace=False)

        for user in tqdm(sampled_users, desc="ROC-AUC"):
            # Positive scores (test items)
            test_items = self.test_matrix.rows[user]
            if not test_items:
                continue

            # Take all test items as positive
            for pos_item in test_items:
                pos_score = torch.dot(user_emb[user], item_emb[pos_item]).item()
                scores.append(pos_score)
                labels.append(1)

            # Negative scores (not in train and not in test)
            train_items = self.train_items[user]
            test_set = set(test_items)
            forbidden = train_items.union(test_set)

            # Find negative items
            attempts = 0
            neg_samples_needed = len(test_items)  # Balance positive and negative
            while attempts < 100 and len(scores) < labels.count(1) + neg_samples_needed:
                neg_item = np.random.randint(0, self.num_items)
                if neg_item not in forbidden:
                    neg_score = torch.dot(user_emb[user], item_emb[neg_item]).item()
                    scores.append(neg_score)
                    labels.append(0)
                attempts += 1

        try:
            roc_auc = roc_auc_score(labels, scores)
        except:
            roc_auc = 0.5

        return roc_auc

In [ ]:
# Cell 8: Trainer with early stopping
class Trainer:
    def __init__(self, model, train_loader, config, metrics_calculator):
        self.model = model
        self.train_loader = train_loader
        self.config = config
        self.metrics_calculator = metrics_calculator
        self.device = next(model.parameters()).device

        # Optimizer WITHOUT weight_decay on embeddings
        self.optimizer = torch.optim.Adam(
            model.parameters(),
            lr=config['learning_rate']
            # NO weight_decay! LightGCN doesn't use L2 regularization
        )

        self.criterion = BPRLoss()  # Without regularization

        # For early stopping
        self.best_hr = 0
        self.patience_counter = 0
        self.best_epoch = 0

        self.train_losses = []
        self.val_metrics_history = []

    def train_epoch(self, adjacency_matrix):
        """One training epoch"""
        self.model.train()
        total_loss = 0
        num_batches = 0

        progress_bar = tqdm(self.train_loader, desc="Training")

        for batch in progress_bar:
            users = batch['user'].to(self.device)
            pos_items = batch['pos_item'].to(self.device)
            neg_items = batch['neg_item'].to(self.device)  # Only 1 negative!

            # Get embeddings
            user_emb, item_emb = self.model(adjacency_matrix)

            # Embeddings for batch
            batch_user_emb = user_emb[users]
            batch_pos_emb = item_emb[pos_items]
            batch_neg_emb = item_emb[neg_items]

            # BPR loss with 1 negative example
            loss = self.criterion(batch_user_emb, batch_pos_emb, batch_neg_emb)

            # Optimization
            self.optimizer.zero_grad()
            loss.backward()

            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

            self.optimizer.step()

            total_loss += loss.item()
            num_batches += 1

            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'avg_loss': f'{total_loss/num_batches:.4f}'
            })

        return total_loss / num_batches

    def validate(self, adjacency_matrix, epoch):
        """Validate the model"""
        print("\nValidation...")

        # Calculate all metrics
        metrics = self.metrics_calculator.calculate_all_metrics(
            self.model, adjacency_matrix, batch_size=4096
        )

        hr = metrics[f'hr@{self.metrics_calculator.k}']

        print(f"  Epoch {epoch}:")
        for metric_name, metric_value in metrics.items():
            print(f"  {metric_name}: {metric_value:.4f}")

        # Early stopping
        if hr > self.best_hr:
            self.best_hr = hr
            self.best_epoch = epoch
            self.patience_counter = 0

            # Save best model
            torch.save({
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'hr': hr,
                'ndcg': metrics[f'ndcg@{self.metrics_calculator.k}'],
                'precision': metrics[f'precision@{self.metrics_calculator.k}'],
                'recall': metrics[f'recall@{self.metrics_calculator.k}'],
                'roc_auc': metrics['roc_auc']
            }, 'best_model.pth')

            print(f"  Saved best model! HR@{self.metrics_calculator.k}: {hr:.4f}")
        else:
            self.patience_counter += 1
            print(f"  Patience: {self.patience_counter}/{self.config['patience']}")

        metrics['epoch'] = epoch
        self.val_metrics_history.append(metrics)

        return metrics

    def train(self, adjacency_matrix, epochs):
        """Full training cycle with early stopping"""
        print(f"Starting training for {epochs} epochs")
        print(f"Early stopping after {self.config['patience']} epochs without improvement")
        start_time = time.time()

        for epoch in range(1, epochs + 1):
            print(f"\nEpoch {epoch}/{epochs}")
            print("-" * 50)

            # Training
            train_loss = self.train_epoch(adjacency_matrix)
            self.train_losses.append(train_loss)

            # Validation
            val_metrics = self.validate(adjacency_matrix, epoch)

            # Check early stopping
            if self.patience_counter >= self.config['patience']:
                print(f"\nEarly stopping at epoch {epoch}")
                print(f"Best HR@{self.metrics_calculator.k}: {self.best_hr:.4f} (epoch {self.best_epoch})")
                break

            # Print statistics
            epoch_time = time.time() - start_time
            print(f"Epoch time: {epoch_time:.1f} sec")
            print(f"Best HR@{self.metrics_calculator.k}: {self.best_hr:.4f}")

        total_time = time.time() - start_time
        print(f"\nTraining completed in {total_time:.1f} sec")
        print(f"Total epochs: {epoch}")
        print(f"Final best HR@{self.metrics_calculator.k}: {self.best_hr:.4f} (epoch {self.best_epoch})")

        return self.best_hr

In [ ]:
# Cell 9: Main function
def main():
    # CONFIGURATION according to LightGCN recommendations
    config = {
        'embedding_dim': 64,          # LightGCN standard: 64 (not more!)
        'num_layers': 3,              # 2-4 layers optimal
        'learning_rate': 1e-3,        # Standard learning rate
        'weight_decay': 0.0,          # IMPORTANT: LightGCN does NOT use L2 regularization!
        'batch_size': 2048,           # Optimal batch size
        'epochs': 5,                # LightGCN trains slowly (hundreds of epochs)
        'patience': 20,               # Early stopping
        'k': 50                       # For @K metrics
    }

    print("LightGCN Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")

    # 1. Load data
    print("\n1. Loading data...")
    data_loader = PreprocessedDataLoader('/kaggle/input/dkr2025/')
    data = data_loader.load_preprocessed_data()

    num_users = len(data['user_to_idx'])
    num_movies = len(data['movie_to_idx'])

    # 2. Prepare normalized adjacency matrix for LightGCN
    print("\n2. Preparing graph...")
    adj_matrix = data['adj_matrix']

    # Normalization as in original LightGCN
    adj_matrix = adj_matrix.tolil()
    adj_matrix = adj_matrix + sp.eye(adj_matrix.shape[0])  # Add self-loop

    rowsum = np.array(adj_matrix.sum(1)).flatten()
    d_inv_sqrt = np.power(rowsum, -0.5)
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

    norm_adj = d_mat_inv_sqrt.dot(adj_matrix).dot(d_mat_inv_sqrt)
    norm_adj_coo = norm_adj.tocoo()

    # Create SparseTensor
    adjacency_matrix = SparseTensor(
        row=torch.tensor(norm_adj_coo.row, dtype=torch.long),
        col=torch.tensor(norm_adj_coo.col, dtype=torch.long),
        value=torch.tensor(norm_adj_coo.data, dtype=torch.float32),
        sparse_sizes=(norm_adj_coo.shape[0], norm_adj_coo.shape[1])
    )

    # 3. Create datasets and loaders
    print("\n3. Preparing training data...")
    train_dataset = MovieLensDataset(
        data['train_ratings'],
        num_users,
        num_movies  # Only 1 negative by default
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=0,  # Avoid multiprocessing issues
        pin_memory=True
    )

    # 4. Create model
    print("\n4. Creating LightGCN model...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    model = LightGCN(
        num_users=num_users,
        num_items=num_movies,
        embedding_dim=config['embedding_dim'],
        num_layers=config['num_layers']
    ).to(device)

    adjacency_matrix = adjacency_matrix.to(device)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # 5. Create metrics calculator
    metrics_calculator = MetricsCalculator(
        train_matrix=data['train_matrix'],
        test_matrix=data['test_matrix'],
        num_users=num_users,
        num_items=num_movies,
        device=device,
        k=config['k']
    )

    # 6. Training
    print("\n5. Training model...")
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        config=config,
        metrics_calculator=metrics_calculator
    )

    best_hr = trainer.train(adjacency_matrix, config['epochs'])

    # 7. Final testing
    print("\n6. Final testing...")
    if os.path.exists('best_model.pth'):
        # Fix loading error
        try:
            checkpoint = torch.load('best_model.pth', weights_only=False)
        except:
            # Alternative way
            checkpoint = torch.load('best_model.pth', weights_only=False, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Loaded best model from epoch {checkpoint['epoch']}")
        print(f"Best metrics:")
        print(f"  HR@{config['k']}: {checkpoint['hr']:.4f}")
        print(f"  NDCG@{config['k']}: {checkpoint['ndcg']:.4f}")
        print(f"  Precision@{config['k']}: {checkpoint['precision']:.4f}")
        print(f"  Recall@{config['k']}: {checkpoint['recall']:.4f}")
        print(f"  ROC-AUC: {checkpoint['roc_auc']:.4f}")

    # Final metrics
    final_metrics = metrics_calculator.calculate_all_metrics(
        model, adjacency_matrix, batch_size=4096
    )

    print("\n" + "="*60)
    print("FINAL RESULTS:")
    print("="*60)
    for metric_name, metric_value in final_metrics.items():
        print(f"{metric_name:20} {metric_value:.4f}")
    print(f"{'Best HR during training:':20} {best_hr:.4f}")
    print("="*60)

    # Save metrics history
    history_df = pd.DataFrame(trainer.val_metrics_history)
    history_df.to_csv('training_history.csv', index=False)
    print("\nTraining history saved to training_history.csv")

if __name__ == "__main__":
    main()

LightGCN Configuration:
  embedding_dim: 64
  num_layers: 3
  learning_rate: 0.001
  weight_decay: 0.0
  batch_size: 2048
  epochs: 5
  patience: 20
  k: 50

1. Loading data...
Loading preprocessed data...
Data successfully loaded!
Users: 162,541
Movies: 59,047
Training interactions: 20,000,076
Test interactions: 610,664

2. Preparing graph...

3. Preparing training data...
Positive interactions: 12,306,450
Using 1 negative example per positive

4. Creating LightGCN model...
Device: cuda
Model parameters: 14,181,632

5. Training model...
Starting training for 5 epochs
Early stopping after 20 epochs without improvement

Epoch 1/5
--------------------------------------------------


Training:   0%|          | 0/6010 [00:00<?, ?it/s]


Validation...
Calculating metrics for 6342 users...


Calculating ranking metrics:   0%|          | 0/2 [00:00<?, ?it/s]

ROC-AUC:   0%|          | 0/5000 [00:00<?, ?it/s]

...

ROC-AUC:   0%|          | 0/5000 [00:00<?, ?it/s]

  Epoch 5:
  hr@50: 0.8147
  ndcg@50: 0.1942
  precision@50: 0.1469
  recall@50: 0.1316
  roc_auc: 0.9389
  Saved best model! HR@50: 0.8147
Epoch time: 11124.7 sec
Best HR@50: 0.8147

Training completed in 11124.7 sec
Total epochs: 5
Final best HR@50: 0.8147 (epoch 5)

6. Final testing...
Loaded best model from epoch 5
Best metrics:
  HR@50: 0.8147
  NDCG@50: 0.1942
  Precision@50: 0.1469
  Recall@50: 0.1316
  ROC-AUC: 0.9389
Calculating metrics for 6342 users...


Calculating ranking metrics:   0%|          | 0/2 [00:00<?, ?it/s]

ROC-AUC:   0%|          | 0/5000 [00:00<?, ?it/s]


FINAL RESULTS:
hr@50                0.8147
ndcg@50              0.1942
precision@50         0.1469
recall@50            0.1316
roc_auc              0.9462
Best HR during training: 0.8147

Training history saved to training_history.csv


In [ ]:
# Cell 10: Quick test
def quick_test():
    """Quick test of one batch"""
    print("="*60)
    print("QUICK MODEL TEST")
    print("="*60)

    # Load minimal data for test
    data_loader = PreprocessedDataLoader('/kaggle/input/dkr2025/')
    data = data_loader.load_preprocessed_data()

    num_users = len(data['user_to_idx'])
    num_movies = len(data['movie_to_idx'])

    # Create mini-dataset
    test_df = data['train_ratings'].head(1000)
    test_dataset = MovieLensDataset(test_df, num_users, num_movies)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Create model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LightGCN(num_users, num_movies, embedding_dim=64, num_layers=3).to(device)

    # Test one batch
    batch = next(iter(test_loader))

    print(f"Batch size: {len(batch['user'])}")
    print(f"Users: {batch['user'][:5].tolist()}")
    print(f"Pos items: {batch['pos_item'][:5].tolist()}")
    print(f"Neg items: {batch['neg_item'][:5].tolist()}")

    # Forward pass
    model.eval()
    with torch.no_grad():
        # Prepare adjacency matrix
        adj_matrix = data['adj_matrix'].tocoo()
        adjacency_matrix = SparseTensor(
            row=torch.tensor(adj_matrix.row, dtype=torch.long),
            col=torch.tensor(adj_matrix.col, dtype=torch.long),
            value=torch.tensor(adj_matrix.data, dtype=torch.float32),
            sparse_sizes=(adj_matrix.shape[0], adj_matrix.shape[1])
        ).to(device)

        user_emb, item_emb = model(adjacency_matrix)

        users = batch['user'].to(device)
        pos_items = batch['pos_item'].to(device)
        neg_items = batch['neg_item'].to(device)

        # Calculate scores
        pos_scores = (user_emb[users] * item_emb[pos_items]).sum(dim=1)
        neg_scores = (user_emb[users] * item_emb[neg_items]).sum(dim=1)

        print(f"\nPositive scores: {pos_scores[:5].cpu().numpy()}")
        print(f"Negative scores: {neg_scores[:5].cpu().numpy()}")
        print(f"Mean difference: {pos_scores.mean().item() - neg_scores.mean().item():.4f}")

        # Simple check
        if pos_scores.mean() > neg_scores.mean():
            print("Model correctly distinguishes positive from negative!")
        else:
            print("Model doesn't distinguish positive and negative examples")

    print("\nTest completed!")

# Run quick test if needed
# quick_test()

In [ ]:
# Cell 11: User Analysis and Recommendations
import pandas as pd

def load_movie_metadata():
    """Load movie metadata for title mapping"""
    try:
        # Load movies data
        movies_df = pd.read_csv("/kaggle/input/movie-lens25/movies.csv")

        # Create mapping from movieId to title
        movieId_to_title = dict(zip(movies_df["movieId"], movies_df["title"]))

        print(f"Loaded metadata for {len(movieId_to_title)} movies")
        return movieId_to_title
    except Exception as e:
        print(f"Could not load movie metadata: {e}")
        return {}

def create_reverse_mappings(user_to_idx, movie_to_idx):
    """Create reverse mappings from internal indices to original IDs"""
    idx_to_user = {v: k for k, v in user_to_idx.items()}
    idx_to_movie = {v: k for k, v in movie_to_idx.items()}
    return idx_to_user, idx_to_movie

def get_top_rated_movies(user_id_internal, ratings_df, idx_to_user, movie_to_idx, movieId_to_title, top_n=10):
    """Return the user's top-N highest rated movies using ratings.csv."""
    # Convert internal id → real MovieLens userId
    if user_id_internal not in idx_to_user:
        print(f"User ID {user_id_internal} not found in mappings")
        return []

    real_user_id = idx_to_user[user_id_internal]

    # Get all user ratings
    user_ratings = ratings_df[ratings_df["userId"] == real_user_id]

    if user_ratings.empty:
        print(f"No ratings found for user {real_user_id}")
        return []

    # Sort by rating descending
    user_ratings = user_ratings.sort_values("rating", ascending=False)

    # Take top N
    top = user_ratings.head(top_n)

    favorites = []
    for _, row in top.iterrows():
        movieId = row["movieId"]
        rating = row["rating"]
        timestamp = row.get("timestamp", None)

        # Map movieId → internal item index (if exists)
        if movieId in movie_to_idx:
            item_idx = movie_to_idx[movieId]
            title = movieId_to_title.get(movieId, f"Unknown {movieId}")
            favorites.append({
                'item_idx': item_idx,
                'title': title,
                'rating': rating,
                'timestamp': timestamp
            })
        else:
            # Movie not in our dataset
            favorites.append({
                'item_idx': None,
                'title': movieId_to_title.get(movieId, f"Unknown {movieId}"),
                'rating': rating,
                'timestamp': timestamp
            })

    return favorites

def recommend_for_user_lightgcn(model, adjacency_matrix, user_id_internal, train_matrix,
                              idx_to_movie, movieId_to_title, device, k=20):
    """Generate recommendations for a specific user using LightGCN model."""
    model.eval()

    num_users = model.num_users
    num_items = model.num_items

    # -----------------------------------------------------
    # 2. Model predictions for unseen items
    # -----------------------------------------------------
    with torch.no_grad():
        # Get embeddings
        user_emb, item_emb = model(adjacency_matrix)

        # Calculate scores for all items
        u_emb = user_emb[user_id_internal]
        scores = torch.matmul(u_emb, item_emb.T)

        # Mask seen interactions
        if hasattr(train_matrix, 'rows'):
            # For LIL format
            seen_items = set(train_matrix.rows[user_id_internal])
        else:
            # For CSR format
            seen_items = set(train_matrix[user_id_internal].nonzero()[1])

        for iid in seen_items:
            scores[iid] = -float('inf')

        # Get top-k recommendations
        topk_values, topk_indices = torch.topk(scores, min(k, len(scores)))
        top_items = topk_indices.cpu().numpy()
        top_scores = topk_values.cpu().numpy()

    # Convert to recommendations
    recommendations = []
    for iid, score in zip(top_items, top_scores):
        if iid in idx_to_movie:
            real_movie_id = idx_to_movie[iid]
            title = movieId_to_title.get(real_movie_id, f"Unknown {real_movie_id}")
            recommendations.append({
                'item_idx': iid,
                'title': title,
                'score': float(score)
            })
        else:
            recommendations.append({
                'item_idx': iid,
                'title': f"Item {iid}",
                'score': float(score)
            })

    return recommendations

def analyze_user_recommendations(user_id_internal, model, adjacency_matrix, train_matrix,
                                user_to_idx, movie_to_idx, ratings_df, device, k=20):
    """Complete analysis for a user: top rated movies and recommendations."""

    # Load movie metadata
    movieId_to_title = load_movie_metadata()

    # Create reverse mappings
    idx_to_user, idx_to_movie = create_reverse_mappings(user_to_idx, movie_to_idx)

    print(f"\n{'='*60}")
    print(f"ANALYSIS FOR USER (internal ID: {user_id_internal})")
    print(f"Real user ID: {idx_to_user.get(user_id_internal, 'Unknown')}")
    print(f"{'='*60}")

    # 1. Get user's top rated movies
    if ratings_df is not None:
        favorites = get_top_rated_movies(
            user_id_internal, ratings_df, idx_to_user,
            movie_to_idx, movieId_to_title, top_n=10
        )

        if favorites:
            print(f"\n=== Top {len(favorites)} Highest-Rated Movies by User ===")
            for i, fav in enumerate(favorites, 1):
                if fav['item_idx'] is not None:
                    print(f"{i:2}. {fav['title']} — rated {fav['rating']:.1f}")
                else:
                    print(f"{i:2}. {fav['title']} — rated {fav['rating']:.1f} (not in training data)")
        else:
            print("\nNo rating history found for this user")

    # 2. Get recommendations
    recommendations = recommend_for_user_lightgcn(
        model, adjacency_matrix, user_id_internal,
        train_matrix, idx_to_movie, movieId_to_title, device, k=k
    )

    print(f"\n=== Model Recommendations (Top {len(recommendations)}) ===")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i:2}. {rec['title']} — score {rec['score']:.4f}")

    return favorites if 'favorites' in locals() else None, recommendations

def analyze_multiple_users(model, adjacency_matrix, train_matrix, user_to_idx,
                          movie_to_idx, ratings_df, device, user_ids=None, k=20):
    """Analyze recommendations for multiple users."""
    if user_ids is None:
        # Analyze users with test interactions (from your metrics calculator)
        num_users = len(user_to_idx)
        # Sample 5 random users
        user_ids = np.random.choice(num_users, size=min(5, num_users), replace=False)

    all_results = {}
    for user_id in user_ids:
        try:
            favorites, recommendations = analyze_user_recommendations(
                user_id, model, adjacency_matrix, train_matrix,
                user_to_idx, movie_to_idx, ratings_df, device, k
            )
            all_results[user_id] = {
                'favorites': favorites,
                'recommendations': recommendations
            }
        except Exception as e:
            print(f"Error analyzing user {user_id}: {e}")

    return all_results

def save_recommendations_to_csv(recommendations, filename="user_recommendations.csv"):
    """Save recommendations to CSV file."""
    if not recommendations:
        print("No recommendations to save")
        return

    # Flatten the data
    rows = []
    for user_id, data in recommendations.items():
        if data['recommendations']:
            for rec in data['recommendations']:
                rows.append({
                    'user_id': user_id,
                    'item_id': rec['item_idx'],
                    'title': rec['title'],
                    'score': rec['score']
                })

    if rows:
        df = pd.DataFrame(rows)
        df.to_csv(filename, index=False)
        print(f"Saved {len(df)} recommendations to {filename}")
    else:
        print("No valid recommendations to save")

In [ ]:
# Cell 12: Demo - Analyze recommendations for sample users
def demo_recommendations():
    """Demonstrate recommendation analysis"""
    print("Loading data for recommendation analysis...")

    # Reload data if needed
    data_loader = PreprocessedDataLoader('/kaggle/input/dkr2025/')
    data = data_loader.load_preprocessed_data()

    # Load ratings for user history
    try:
        ratings_df = pd.read_csv("/kaggle/input/dkr2025/ratings.csv")
        print(f"Loaded ratings data: {len(ratings_df):,} ratings")
    except:
        print("Could not load ratings.csv, using train_ratings instead")
        ratings_df = data['train_ratings']
        ratings_df = ratings_df.rename(columns={'user_idx': 'userId', 'movie_idx': 'movieId'})

    # Load best model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if os.path.exists('best_model.pth'):
        try:
            checkpoint = torch.load('best_model.pth', weights_only=False, map_location=device)

            # Recreate model
            num_users = len(data['user_to_idx'])
            num_movies = len(data['movie_to_idx'])

            model = LightGCN(
                num_users=num_users,
                num_items=num_movies,
                embedding_dim=64,
                num_layers=3
            ).to(device)

            model.load_state_dict(checkpoint['model_state_dict'])
            print(f"Loaded best model from epoch {checkpoint['epoch']}")

            # Prepare adjacency matrix
            adj_matrix = data['adj_matrix'].tocoo()
            adjacency_matrix = SparseTensor(
                row=torch.tensor(adj_matrix.row, dtype=torch.long),
                col=torch.tensor(adj_matrix.col, dtype=torch.long),
                value=torch.tensor(adj_matrix.data, dtype=torch.float32),
                sparse_sizes=(adj_matrix.shape[0], adj_matrix.shape[1])
            ).to(device)

            # Analyze some users
            print("\n" + "="*60)
            print("DEMONSTRATING RECOMMENDATIONS")
            print("="*60)

            # Sample some users to analyze
            sample_users = np.random.choice(num_users, size=min(3, num_users), replace=False)

            results = analyze_multiple_users(
                model=model,
                adjacency_matrix=adjacency_matrix,
                train_matrix=data['train_matrix'],
                user_to_idx=data['user_to_idx'],
                movie_to_idx=data['movie_to_idx'],
                ratings_df=ratings_df,
                device=device,
                user_ids=sample_users,
                k=15
            )

            # Save results
            save_recommendations_to_csv(results, "sample_recommendations.csv")

            print("\n" + "="*60)
            print("DEMONSTRATION COMPLETE")
            print("="*60)

        except Exception as e:
            print(f"Error in demo: {e}")
    else:
        print("No trained model found. Please train the model first.")

# Run the demo
demo_recommendations()